In [1]:
import pImpactR as impact
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as copy
rectangular_ = 1
elliptic_ = 2

In [2]:
eSpread = 0.0# 2.0e-3
NL_t = 0.4
CLcut = 2.6


emitGeomRMS = 3.3e-6
npt = 100000
print('npt=',npt)
nturn = 1

npt= 100000


In [3]:
NL_L = 1.8
NL_c = 0.01
NL_nu = 0.3

betx = NL_L/np.sin(2.0*np.pi*NL_nu)
alfx = np.tan(np.pi*NL_nu)

# read Aperture

In [4]:
aperture = np.loadtxt('../pipeinfo.in')

# read impact lattice

In [5]:
beam,lattice = impact.readInputFile('../test.Chad.in')
beam.nCore_y=2
beam.nCore_z=5
beam.n_particles = npt

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

g = ke/mass+1.0
bg = np.sqrt(g**2-1.0)
emitN = emitGeomRMS*bg

reading ImpactZ input file (../test.Chad.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [6]:
cleanLat = impact.clearLattice(lattice)
L = 0 
for item in cleanLat:
    if item.type == 'RFkick':
        item.vmax = 0.0
    if item.type == 'nonlinear_insert':
        print('NL:',L,L+item.length)
    if 'length' in item:
        L = L+item.length
print(L-aperture[-1,0])

NL: 4.9115191429 6.711519142899999
-4.199918635094946e-09


In [7]:
for item in cleanLat:
    if 'length' in item:
        item.n_sckick = int(np.ceil(item.length*50))
        item.n_map = 1

# Back tracking for IOTA matched beam at injection location

In [8]:
for i in range(len(cleanLat)):
    if cleanLat[i].type == 'nonlinear_insert':
        break

latticeB = impact.getInverseLattice(cleanLat[:i])

In [9]:
write0 = impact.getElem('write_raw_ptcl')
write0.file_id = 111110
write0.format_id = 2
latticeB.insert(0,write0)

write0 = impact.getElem('write_raw_ptcl')
write0.file_id = 222220
write0.format_id = 1
latticeB.insert(0,write0)


write1 = impact.getElem('write_raw_ptcl')
write1.file_id = 111111
write1.format_id = 2
latticeB.append(write1)

write1 = impact.getElem('write_raw_ptcl')
write1.file_id = 222221
write1.format_id = 1
latticeB.append(write1)

In [10]:
loop = impact.getElem('loop')
loop.turns = 1
latticeB.insert(0,loop)

In [11]:
beam.distribution.distribution_type = 'IOTA_Gauss'
beam.distribution.NL_t  = NL_t
beam.distribution.NL_c  = NL_c
beam.distribution.betx  = betx
beam.distribution.alfx  = alfx
beam.distribution.emitx = emitGeomRMS
beam.distribution.CL    = CLcut
beam.distribution.betz  = 200
beam.distribution.alfz  = 0.0
beam.distribution.emitz = (eSpread*ke*1.0e-6)**2*beam.distribution.betz
betx,alfx,emitGeomRMS,beam.distribution.emitz

(1.8926320036288808, 1.3763819204711734, 3.3e-06, 0.0)

In [12]:
impact.writeInputFile(beam,latticeB)
impact.run(beam,order=1)

0

In [13]:
pData0 = impact.readParticleData(111110,ke,mass,freq,2)
pData00 = impact.readParticleData(222220,ke,mass,freq,1)

In [14]:
impact.getTwiss_from_pData(pData0)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: invalid value encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(1.471202808910873,
 1.077753089824901,
 1.7405779452141819e-06,
 3.390449912881651,
 2.468893062770939,
 4.021952908813562e-06,
 nan,
 nan,
 0.0)

In [15]:
impact.getTwiss_from_pData(pData00)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: invalid value encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(1.471202808910873,
 1.077753089824901,
 1.7405779452141819e-06,
 3.390449912881651,
 2.468893062770939,
 4.021952908813562e-06,
 nan,
 nan,
 0.0)

In [16]:
pData1 = impact.readParticleData(111111,ke,mass,freq,2)
pData11 = impact.readParticleData(222221,ke,mass,freq,1)

In [17]:
impact.getTwiss_from_pData(pData1)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: divide by zero encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(0.4808181886655089,
 0.18515367518612716,
 1.7405779452141761e-06,
 2.106761738737537,
 0.5102521351291409,
 4.021952908813548e-06,
 inf,
 nan,
 0.0)

In [18]:
impact.getTwiss_from_pData(pData11)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: divide by zero encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(0.4808181886655089,
 0.18515367518612716,
 1.740577945214176e-06,
 2.106761738737537,
 0.5102521351291409,
 4.021952908813548e-06,
 inf,
 nan,
 0.0)

In [19]:
np.savetxt('rays.in',pData1[:,:6])

# run

In [20]:
beam.distribution.distribution_type = 'ReadFile'
impact.writeParticleData(pData1,ke,mass,freq)

In [21]:
# beam.distribution.alfx = 3.860846841e-5
# beam.distribution.betx = 0.561065822
# beam.distribution.alfy = 2.282987698e-9
# beam.distribution.bety = 1.385141718
# beam.distribution.emitx = emitN
# beam.distribution.emity = emitN
# beam.distribution.betz  = 200
# beam.distribution.alfz  = 0.0
# beam.distribution.emitz = (eSpread*ke*1.0e-6)**2*beam.distribution.betz
# beam.distribution.distribution_type = 'IOTA_Gauss'

# run Impact

In [22]:
cleanLat.insert(0,impact.getElem('pipeinfo'))
cleanLat.insert(1,impact.getElem('loop'))
cleanLat[1].turns = nturn

In [23]:
for i in range(len(cleanLat)):
    if cleanLat[i].type == 'nonlinear_insert':
        break
    
writeP = impact.getElem('write_raw_ptcl')
writeP.file_id = 1001
writeP.format_id = 1
cleanLat.insert(i,copy(writeP))

writeP.file_id = 1002
cleanLat.insert(i+2,copy(writeP))


writeP.file_id = 1003
cleanLat.append(copy(writeP))

writeP.file_id = 1000
cleanLat.insert(2,copy(writeP))

cleanLat.pop(0)


 type: 'pipeinfo'

In [24]:
cleanLat = impact.addHardEdgeQuad(cleanLat)
impact.writeInputFile(beam,cleanLat)
impact.run(beam,order=1)

0

In [25]:
pData3 = impact.readParticleData(1003,ke,mass,freq,1)

In [26]:
impact.getTwiss_from_pData(pData1)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: divide by zero encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(0.4808181886655089,
 0.18515367518612716,
 1.7405779452141761e-06,
 2.106761738737537,
 0.5102521351291409,
 4.021952908813548e-06,
 inf,
 nan,
 0.0)

In [27]:
impact.getTwiss_from_pData(pData3)

/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1658: RuntimeWarning: divide by zero encountered in double_scalars
  betz = varx/emitz
/users/kilean/IOTA/pImpactR/pImpactR/impactIO.py:1659: RuntimeWarning: invalid value encountered in double_scalars
  alfz = np.sqrt(varpx/emitz*betz-1)


(0.48469670689780336,
 0.17717436196675504,
 1.7395473061767852e-06,
 2.116780725667204,
 0.4962195908230349,
 4.022955043431966e-06,
 inf,
 nan,
 0.0)